In [4]:
import pandas as pd

df = pd.read_csv("products.csv")

In [5]:
df.shape

(30856, 7)

In [6]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30856 entries, 0 to 30855
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   sku                30856 non-null  object
 1   categories         23893 non-null  object
 2   name               27100 non-null  object
 3   description        23738 non-null  object
 4   short_description  15556 non-null  object
 5   price              26899 non-null  object
 6   url_key            26905 non-null  object
dtypes: object(7)
memory usage: 1.6+ MB


In [7]:
# SKU is useless
df = df.drop(columns=["sku"])

In [8]:
df = df.dropna()
df = df.drop_duplicates()

In [9]:
df.reset_index(inplace=True, drop=True)


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13424 entries, 0 to 13423
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   categories         13424 non-null  object
 1   name               13424 non-null  object
 2   description        13424 non-null  object
 3   short_description  13424 non-null  object
 4   price              13424 non-null  object
 5   url_key            13424 non-null  object
dtypes: object(6)
memory usage: 629.4+ KB


In [11]:
df['categories'][3343]
def split_categories(series : pd.Series) -> pd.DataFrame:
    # Remove 'Default Category/', split by '/', and expand into columns
    split_cols = series.str.strip(r"/").replace(r'^Default Category/', '', regex=True).str.split('/', expand=True)
    # Rename columns dynamically as SubCat_1, SubCat_2, ...
    split_cols.columns = [f'SubCat_{i+1}' for i in range(split_cols.shape[1])]
    return split_cols

def split_categories_reverse(series: pd.Series) -> pd.DataFrame:
    # Remove 'Default Category/', split by '/', and reverse the list
    split_cols = (
        series.str.replace(r'^Default Category/', '', regex=True)
              .str.split('/')
              .apply(lambda parts: parts[::-1])  # reverse the list
    )
    # Find the max number of subcategories
    max_len = split_cols.apply(len).max()
    
    # Create new DataFrame with reversed subcategories
    return pd.DataFrame(split_cols.tolist(), columns=[f'SubCat_{i+1}' for i in range(max_len)])

# df['categories'].map(split_categories)
subcats = split_categories_reverse(df['categories'])[['SubCat_1', "SubCat_2"]]
for col in subcats.columns:
    # print(col)
    # if col.startswith(""):
    print(subcats[col].isna().sum())


0
326


In [12]:

df = pd.concat([df, subcats.apply(lambda x: x.str.lower())], axis=1).dropna()

In [13]:
final_cats = df.SubCat_2.str.lower().str.replace(",default category", "").value_counts().iloc[:30].index.tolist()
final_cats = [cat.replace(" ", "_") for cat in final_cats]

In [14]:
df = df[df["SubCat_2"].str.lower().isin(final_cats)].reset_index(drop=True)

In [15]:
df = df.drop(["categories", "SubCat_1"], axis=1).rename({"SubCat_2": "category"}, axis=1)

In [16]:
# Clean description

from bs4 import BeautifulSoup

def clean_html(text):
    # Parse the HTML content
    soup = BeautifulSoup(text, "html.parser")
    # Extract text from the parsed HTML
    return soup.get_text(separator="\n", strip=True)

df['description'] = df['description'].apply(clean_html)
df['short_description'] = df['short_description'].apply(clean_html)


In [17]:
df['url_key'] = "https://huntersunite.com/"+ df['url_key']


In [18]:
df.shape

(7510, 6)

In [19]:
# df.to_csv("products_cleaned.csv", index=False)